In [ ]:
import numpy as np
import torch.nn.functional as nnFunctions
from torch.autograd import Variable
import torch
from torch import nn

In [ ]:
import torchvision

In [ ]:
from numpy import newaxis

# TempTest Data

a=np.zeros((1,512,512)).astype(np.float32)
b=np.ones((1,512,512)).astype(np.float32)

a=a[...,newaxis]
b=b[...,newaxis]
a=np.transpose(a,[0,3,1,2])
b=np.transpose(a,[0,3,1,2])

In [ ]:
np_data=np.fromfile('trainImagesNumpy_512x512_color.txt')

In [ ]:
np_data.shape

In [ ]:
np_data=np_data.reshape(410,512,512,-1).astype(np.float32)

In [ ]:
np_data=np.transpose(np_data,[0,3,1,2])

In [ ]:
np_data.shape

In [ ]:
np_labels=np.fromfile('trainMaskNumpy_512x512.txt')

In [ ]:
np_labels.shape

In [ ]:
np_labels=np_labels.reshape(410,512,512)

In [ ]:
np_labels=np_labels.astype(np.float32)[...,newaxis]
np_labels=np.transpose(np_labels,[0,3,1,2])

In [ ]:
np_labels.shape

# Features and Label Tensors

In [ ]:
features=torch.from_numpy(np_data)
targets=torch.from_numpy(np_labels)

In [ ]:
batch_size=5

In [ ]:
import torch.utils.data as data_utils

train_data = data_utils.TensorDataset(features, targets)
train_loader = data_utils.DataLoader(train_data, batch_size=batch_size, shuffle=True,num_workers=10)

# Network

In [ ]:
class myConvNet(nn.Module):
    #constructor
    def __init__(self):
        super(myConvNet, self).__init__()
        #defining layers in convnet
        #input size=1*657*1625
        self.conv1 = nn.Conv2d(512,512, kernel_size=3,stride=1,padding=1)
        self.conv2 = nn.Conv2d(512,256, kernel_size=3,stride=1,padding=1)
        #self.bn1=nn.BatchNorm2d(32)
        
        self.conv3 = nn.Conv2d(256,256,kernel_size=3,stride=1,padding=1)
        self.pconv1= nn.Conv2d(256,128, kernel_size=(3,3),stride=1,padding=(1,1))
        #self.bn2=nn.BatchNorm2d(64)
        self.pconv2= nn.Conv2d(256,128, kernel_size=(3,7),stride=1,padding=(1,3))
        self.pconv3= nn.Conv2d(256,128, kernel_size=(7,3),stride=1,padding=(3,1))
        
        self.conv4= nn.Conv2d(128,64,kernel_size=3,stride=1,padding=1)
        self.conv5= nn.Conv2d(64,1,kernel_size=3,stride=1,padding=1)
        
    def forward(self, x,index1,index2,index3,index4):
        x = nnFunctions.max_unpool2d(nnFunctions.leaky_relu(self.conv1(x)),indices=index4,kernel_size=(2,2),stride=(2,2))
        x = nnFunctions.leaky_relu(self.conv2(x))
        x = nnFunctions.max_unpool2d(nnFunctions.leaky_relu(self.conv3(x)),indices=index3,kernel_size=(2,2),stride=(2,2))
        #parallel conv
        x = nnFunctions.max_unpool2d(nnFunctions.leaky_relu(self.pconv1(x)+self.pconv2(x)+self.pconv3(x)),indices=index2,kernel_size=(2,2),stride=(2,2))
        
        x = nnFunctions.max_unpool2d(nnFunctions.leaky_relu(self.conv4(x)),indices=index1,kernel_size=(2,2),stride=(2,2))
        x = nnFunctions.leaky_relu(self.conv5(x))
        return x

In [ ]:
my_net=myConvNet()

In [ ]:
vgg16=torchvision.models.vgg16(pretrained=True)

In [ ]:
modified_pretrained = nn.Sequential(*list(vgg16.features.children())[:-1])

In [ ]:
vgg16

In [ ]:
modified_pretrained

In [ ]:
class finalConvNet(nn.Module):
    global index1,index2,index3,index4
    #constructor
    def __init__(self,mynet,vgg16):
        super(finalConvNet, self).__init__()
        self.vgg=vgg16
        self.custom_net=mynet
        
        for i in range(30):
            if(i<14):
                self.vgg[i].requires_grad=False
            else:
                self.vgg[i].requires_grad=True
            if(i==4):
                self.vgg[i].return_indices=True
            elif(i==9):
                self.vgg[i].return_indices=True
            elif(i==16):
                self.vgg[i].return_indices=True
            elif(i==23):
                self.vgg[i].return_indices=True
        
    def forward(self, x):
        #4 max pool layers
        for layers in range(30):
            if(layers==4):
                x,index1=self.vgg[layers](x)
            elif(layers==9):
                x,index2=self.vgg[layers](x)
            elif(layers==16):
                x,index3=self.vgg[layers](x)
            elif(layers==23):
                x,index4=self.vgg[layers](x)
            else:
                x=self.vgg[layers](x)
        x=self.custom_net(x,index1,index2,index3,index4)
        return x

In [ ]:
net=finalConvNet(my_net,modified_pretrained)

In [ ]:
net.cuda()

In [ ]:
criterion=nn.L1Loss().cuda()

In [ ]:
import torch.optim as optim

In [ ]:
learning_rate=1e-5

In [ ]:
def exp_lr_scheduler(epoch, init_lr=learning_rate, lr_decay_epoch=10):
    """Decay learning rate by a factor of 0.1 every lr_decay_epoch epochs."""
    lr=init_lr
    if epoch % lr_decay_epoch == 0:
        lr = init_lr * (0.1**(epoch // lr_decay_epoch))
        print('LR is set to {}'.format(lr))

    return lr

In [ ]:
def train(train_loader,net,epochs,total_samples,lr_scheduler):
    global learning_rate
    prev_loss=0
    net.train()
    
    
    for epoch in range(int(epochs)): # loop over the dataset multiple times
        running_loss = 0.0
        learning_rate = lr_scheduler(epoch)
        
        optimizer = optim.Adam(net.parameters(),lr=learning_rate)
        
        for i,data in enumerate(train_loader):
            inputs,labels=data
            # wrap them in Variable
            inputs, labels = Variable(inputs).cuda(), Variable(labels).cuda()
            
            # zero the parameter gradients
            optimizer.zero_grad()

            # forward + backward + optimize
            outputs = net(inputs)
            loss = criterion(outputs, labels)
            loss.backward()        
            
            
            optimizer.step()
            # print statistics
            running_loss += loss.data[0]
            cur_loss=loss.data[0]
            print '{0}\r'.format('Batch '+str(i)+':'+str(cur_loss)),
        running_loss=running_loss
        print('\n\t Iteration '+str(epoch)+':'+str(running_loss))
        if(epoch%2==0):
            print('Saving network')
            net.cpu()
            torch.save(net,'Net_checkpoints/net_transfer_learning_4_correct_images_intermediate_'+str(epoch/2)+'.txt')
            net.cuda()
#         if(prev_loss<running_loss):
#             learning_rate/=10
        prev_loss=running_loss
    print('Finished Training')
    return net

In [ ]:
net=train(train_loader,net,10,410,exp_lr_scheduler)

In [ ]:
net.cpu()

In [ ]:
torch.save(net,'net_transfer_learning_5_correct_images.txt')

In [ ]:
net.cuda()